In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline


In [130]:
df_test = pd.read_csv('../Data/test_PCA_jmk_lnd.csv')
df_train = pd.read_csv('../Data/train_PCA_jmk_lnd.csv')
df_valid = pd.read_csv('../Data/valid_PCA_jmk_lnd.csv')
df_te = pd.read_csv('../Data/PJT002_test.csv')
df_tr = pd.read_csv('../Data/PJT002_train.csv')
df_va = pd.read_csv('../Data/PJT002_validation.csv')

In [131]:
df_test.columns

Index(['address', 'address1', 'address2', 'ahsm_dstnc', 'bldng_ar',
       'bldng_cnt', 'bldng_cnt_in_50m', 'cctv_dstnc', 'cctv_in_100m', 'day',
       'dayofweek', 'fire_yes', 'fr_mn_cnt', 'fr_sttn_dstnc',
       'fr_wthr_fclt_dstnc', 'fr_wthr_fclt_in_100m', 'hm_cnt', 'hmdt', 'hour',
       'id', 'jmk', 'lnd_ar', 'minute', 'mlt_us_yn(encode)', 'month',
       'no_tbc_zn_dstnc', 'second', 'sft_emrgnc_bll_dstnc', 'tag',
       'tbc_rtl_str_dstnc', 'tmprtr', 'ttl_ar', 'wnd_drctn', 'wnd_spd', 'year',
       'year-month', 'year-month-day', 'cluster', 'Comp1', 'Comp2', 'jmk_주',
       'jmk_잡', 'jmk_종', 'jmk_차', 'jmk_답', 'jmk_장', 'jmk_양', 'jmk_전', 'jmk_창',
       'jmk_학', 'jmk_목', 'jmk_도', 'jmk_임', 'jmk_철', 'jmk_유', 'jmk_천', 'jmk_구',
       'jmk_체', 'jmk_과', 'jmk_묘', 'jmk_공', 'jmk_원', 'jmk_사', 'jmk_제', 'jmk_수',
       'lnd_us_sttn_nm(clean)', 'lnd_us_sttn_nm_주', 'lnd_us_sttn_nm_잡',
       'lnd_us_sttn_nm_종', 'lnd_us_sttn_nm_차', 'lnd_us_sttn_nm_답',
       'lnd_us_sttn_nm_장', 'lnd_us_sttn_nm_양

In [116]:
# 지하 층 수 null은 평균 0.13... 이므로 0으로 채움
df_test['ttl_dwn_flr'] = df_te['ttl_dwn_flr']
df_test['ttl_dwn_flr'] = df_test['ttl_dwn_flr'].fillna(0)

df_test['ttl_grnd_flr'] = df_te['ttl_grnd_flr']
df_test['ttl_grnd_flr'] = df_test.apply(lambda row: 
                                        1 if (row['ttl_grnd_flr'] == 0) & (row['ttl_dwn_flr'] == 0) & (row['ttl_ar'] != 0) & (row['bldng_ar'] != 0)
                                        else row['ttl_grnd_flr'] if pd.notnull(row['ttl_grnd_flr'])
                                        else 0 if row['ttl_ar'] == 0 or row['bldng_ar'] == 0
                                        else round(row['ttl_ar'] / row['bldng_ar'], 1), 
                                        axis=1)


df_train['ttl_dwn_flr'] = df_tr['ttl_dwn_flr']
df_train['ttl_dwn_flr'] = df_train['ttl_dwn_flr'].fillna(0)

df_train['ttl_grnd_flr'] = df_tr['ttl_grnd_flr']
df_train['ttl_grnd_flr'] = df_train.apply(lambda row:
                                          1 if (row['ttl_grnd_flr'] == 0) & (row['ttl_dwn_flr'] == 0) & (row['ttl_ar'] != 0) & (row['bldng_ar'] != 0)
                                          else row['ttl_grnd_flr'] if pd.notnull(row['ttl_grnd_flr'])
                                          else 0 if row['ttl_ar'] == 0 or row['bldng_ar'] == 0
                                          else round(row['ttl_ar'] / row['bldng_ar'], 1), axis=1)


df_valid['ttl_dwn_flr'] = df_va['ttl_dwn_flr']
df_valid['ttl_dwn_flr'] = df_valid['ttl_dwn_flr'].fillna(0)

df_valid['ttl_grnd_flr'] = df_va['ttl_grnd_flr']

df_valid['ttl_grnd_flr'] = df_valid.apply(lambda row:
                                          1 if (row['ttl_grnd_flr'] == 0) & (row['ttl_dwn_flr'] == 0) & (row['ttl_ar'] != 0) & (row['bldng_ar'] != 0)
                                          else row['ttl_grnd_flr'] if pd.notnull(row['ttl_grnd_flr'])
                                          else 0 if row['ttl_ar'] == 0 or row['bldng_ar'] == 0
                                          else round(row['ttl_ar'] / row['bldng_ar']),
                                          axis=1)

In [117]:
df_test[(df_test['ttl_grnd_flr'] == 0) & (df_test['ttl_ar'] != 0)][['ttl_grnd_flr', 'ttl_dwn_flr', 'ttl_ar', 'bldng_ar']]

,ttl_grnd_flr,ttl_dwn_flr,ttl_ar,bldng_ar
1703,0.0,1.0,11.57,11.57
1929,0.0,0.0,34.24,0.00


In [118]:
df_test.loc[2478, 'ttl_grnd_flr'] = 1

In [119]:
df_valid[(df_valid['ttl_grnd_flr'] == 0) & (df_valid['ttl_ar'] != 0)][['ttl_grnd_flr', 'ttl_dwn_flr', 'ttl_ar', 'bldng_ar']]

,ttl_grnd_flr,ttl_dwn_flr,ttl_ar,bldng_ar
1880,0.0,0.0,132.58,0.0
2846,0.0,0.0,453.20,0.0


In [120]:
df_tr.iloc[644,][['ttl_grnd_flr', 'ttl_dwn_flr', 'ttl_ar', 'bldng_ar']]

ttl_grnd_flr       0
ttl_dwn_flr        0
ttl_ar          37.2
bldng_ar        37.2
Name: 644, dtype: object

In [122]:
df_train[(df_train['ttl_grnd_flr'] == 0) & (df_train['ttl_dwn_flr'] == 0) & (df_train['ttl_ar'] != 0) & (df_train['bldng_ar'] != 0)]['ttl_grnd_flr']

Series([], Name: ttl_grnd_flr, dtype: float64)

In [126]:
df_train[['ttl_grnd_flr', 'ttl_dwn_flr', 'ttl_ar', 'bldng_ar']]

,ttl_grnd_flr,ttl_dwn_flr,ttl_ar,bldng_ar
0,1.0,0.0,112.8600,112.8600
1,5.0,0.0,1109.5400,721.8200
2,1.0,0.0,47.0600,47.0600
3,2.0,0.0,176.6400,171.4500
4,0.0,0.0,0.0000,0.0000
5,200.0,6.0,316147.1200,30351.4800
6,2.0,1.0,337.6100,114.1500
7,1.0,0.0,81.2100,81.2100
8,1.0,0.0,61.9500,61.9500
9,2.0,0.0,254.5300,158.5700


In [125]:
df_test.to_csv('test_PCA_jmk_lnd_fl.csv', index=False)
df_train.to_csv('train_PCA_jmk_lnd_fl.csv', index=False)
df_valid.to_csv('valid_PCA_jmk_lnd_fl.csv', index=False)